In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Load All CSV Files from DemographicData Folder

In [ ]:
# Define the data directory
data_dir = '../DemographicData/'

# Get all CSV files
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

print(f"Total CSV files found: {len(csv_files)}")
print(f"\nFirst 10 files:")
for i, file in enumerate(csv_files[:10], 1):
    print(f"{i}. {os.path.basename(file)}")

In [ ]:
# Load all CSV files and combine them
print("Loading all CSV files...")
print("This may take a few moments...\n")

df_list = []
failed_files = []

for i, file in enumerate(csv_files, 1):
    try:
        df_temp = pd.read_csv(file, low_memory=False)
        df_list.append(df_temp)
        if i % 10 == 0:
            print(f"Loaded {i}/{len(csv_files)} files...")
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Failed to load {os.path.basename(file)}: {e}")

# Combine all dataframes
df_raw = pd.concat(df_list, ignore_index=True)

print(f"\n✓ Successfully loaded {len(df_list)} files")
print(f"✗ Failed to load {len(failed_files)} files")
print(f"\nTotal records loaded: {len(df_raw):,}")

## 2. Verify Data Structure

In [ ]:
# Display basic information
print("Dataset Shape:")
print(f"Rows: {df_raw.shape[0]:,}")
print(f"Columns: {df_raw.shape[1]}")
print("\nColumn Names:")
print(df_raw.columns.tolist())
print("\nData Types:")
print(df_raw.dtypes)
print("\nFirst 10 rows:")
df_raw.head(10)

In [ ]:
# Check for expected columns
expected_columns = ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']
actual_columns = df_raw.columns.tolist()

print("Column Verification:")
print("=" * 50)
for col in expected_columns:
    if col in actual_columns:
        print(f"✓ {col} - Present")
    else:
        print(f"✗ {col} - Missing")

# Check for extra columns
extra_cols = set(actual_columns) - set(expected_columns)
if extra_cols:
    print(f"\nExtra columns found: {list(extra_cols)}")

In [ ]:
# Basic statistics
print("Basic Statistics:")
print("=" * 50)
df_raw.describe(include='all')

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("=" * 50)
missing_counts = df_raw.isnull().sum()
missing_percent = (df_raw.isnull().sum() / len(df_raw)) * 100

missing_df = pd.DataFrame({
    'Column': df_raw.columns,
    'Missing Count': missing_counts,
    'Missing %': missing_percent
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("✓ No missing values found!")

## 3. Date Standardization

In [ ]:
# Create a copy for preprocessing
df = df_raw.copy()

# Check current date format
print("Sample date values before standardization:")
print(df['date'].head(10))
print(f"\nDate data type: {df['date'].dtype}")

In [ ]:
# Convert date to datetime format
print("Converting dates to standardized format...")

# Try different date formats
def parse_date(date_str):
    try:
        # Try DD-MM-YYYY format first
        return pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce')
    except:
        try:
            # Try automatic parsing
            return pd.to_datetime(date_str, errors='coerce')
        except:
            return pd.NaT

df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')

# Check for date parsing failures
date_nulls = df['date'].isnull().sum()
print(f"\nDate parsing results:")
print(f"Successfully parsed: {len(df) - date_nulls:,} ({((len(df) - date_nulls)/len(df)*100):.2f}%)")
print(f"Failed to parse: {date_nulls:,} ({(date_nulls/len(df)*100):.2f}%)")

# Display sample parsed dates
print("\nSample standardized dates:")
print(df['date'].head(10))

In [ ]:
# Extract additional date features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['year_month'] = df['date'].dt.to_period('M')
df['day'] = df['date'].dt.day
df['quarter'] = df['date'].dt.quarter

print("Date features extracted:")
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")
print(f"\nUnique years: {sorted(df['year'].dropna().unique())}")
print(f"Unique months: {sorted(df['month'].dropna().unique())}")
print(f"\nSample with date features:")
df[['date', 'year', 'month', 'quarter', 'year_month']].head()

## 4. District-Pincode Consistency Checks

In [ ]:
# Standardize text columns
print("Standardizing state and district names...")

# Convert to title case and strip whitespace
df['state'] = df['state'].astype(str).str.strip().str.title()
df['district'] = df['district'].astype(str).str.strip().str.title()

# Convert pincode to string and handle invalid values
df['pincode'] = df['pincode'].astype(str).str.strip()

print("✓ Text standardization complete")

In [ ]:
# Unique values check
print("Geographic Data Summary:")
print("=" * 50)
print(f"Unique States: {df['state'].nunique()}")
print(f"Unique Districts: {df['district'].nunique()}")
print(f"Unique Pincodes: {df['pincode'].nunique()}")

print("\nTop 10 States by record count:")
print(df['state'].value_counts().head(10))

In [ ]:
# Create district-pincode mapping for consistency check
district_pincode_map = df.groupby('district')['pincode'].apply(lambda x: set(x)).to_dict()

print(f"Total district-pincode mappings: {len(district_pincode_map)}")
print("\nSample district-pincode relationships:")
for i, (district, pincodes) in enumerate(list(district_pincode_map.items())[:5]):
    print(f"{i+1}. {district}: {len(pincodes)} unique pincodes")
    print(f"   Sample pincodes: {list(pincodes)[:5]}")

In [ ]:
# Check for invalid pincode values
print("Pincode Validation:")
print("=" * 50)

# Valid Indian pincodes are 6-digit numbers
df['pincode_valid'] = df['pincode'].str.match(r'^\d{6}$')

valid_pincodes = df['pincode_valid'].sum()
invalid_pincodes = len(df) - valid_pincodes

print(f"Valid pincodes (6 digits): {valid_pincodes:,} ({(valid_pincodes/len(df)*100):.2f}%)")
print(f"Invalid pincodes: {invalid_pincodes:,} ({(invalid_pincodes/len(df)*100):.2f}%)")

if invalid_pincodes > 0:
    print("\nSample invalid pincodes:")
    print(df[~df['pincode_valid']]['pincode'].value_counts().head(10))

## 5. Missing Value Handling

In [ ]:
# Comprehensive missing value analysis
print("Detailed Missing Value Analysis:")
print("=" * 50)

for col in df.columns:
    null_count = df[col].isnull().sum()
    null_percent = (null_count / len(df)) * 100
    
    if null_count > 0:
        print(f"\n{col}:")
        print(f"  Missing: {null_count:,} ({null_percent:.2f}%)")
        
        # Show sample of rows with missing values
        if null_count <= 10:
            print(f"  Sample rows: {df[df[col].isnull()].index.tolist()[:5]}")

In [ ]:
# Handle missing values
print("Handling Missing Values:")
print("=" * 50)

# Remove rows with missing dates (critical field)
rows_before = len(df)
df = df[df['date'].notna()]
rows_removed_date = rows_before - len(df)
print(f"Removed {rows_removed_date:,} rows with missing dates")

# Fill missing demographic values with 0 (reasonable assumption for update counts)
df['demo_age_5_17'] = df['demo_age_5_17'].fillna(0)
df['demo_age_17_'] = df['demo_age_17_'].fillna(0)
print("Filled missing demographic values with 0")

# Handle missing geographic data
df['state'] = df['state'].fillna('Unknown')
df['district'] = df['district'].fillna('Unknown')
df['pincode'] = df['pincode'].fillna('000000')
print("Filled missing geographic data with placeholders")

print(f"\n✓ Final dataset size: {len(df):,} rows")

## 6. Data Type Conversion & Validation

In [ ]:
# Convert demographic columns to numeric
print("Converting demographic columns to numeric...")

df['demo_age_5_17'] = pd.to_numeric(df['demo_age_5_17'], errors='coerce').fillna(0).astype(int)
df['demo_age_17_'] = pd.to_numeric(df['demo_age_17_'], errors='coerce').fillna(0).astype(int)

print("✓ Demographic columns converted to integers")

# Display data types
print("\nFinal Data Types:")
print(df.dtypes)

In [ ]:
# Check for negative values (data quality check)
print("Data Quality Checks:")
print("=" * 50)

negative_child = (df['demo_age_5_17'] < 0).sum()
negative_adult = (df['demo_age_17_'] < 0).sum()

print(f"Negative values in demo_age_5_17: {negative_child}")
print(f"Negative values in demo_age_17_: {negative_adult}")

# Remove negative values if any
if negative_child > 0 or negative_adult > 0:
    df = df[(df['demo_age_5_17'] >= 0) & (df['demo_age_17_'] >= 0)]
    print(f"✓ Removed rows with negative values")

print(f"\nFinal clean dataset: {len(df):,} rows")

## 7. Monthly Aggregation

In [ ]:
# Create monthly aggregated dataset
print("Creating monthly aggregated dataset...")

# Aggregate by year_month, state, district, pincode
df_monthly = df.groupby(['year_month', 'state', 'district', 'pincode']).agg({
    'demo_age_5_17': 'sum',
    'demo_age_17_': 'sum',
    'date': 'count'  # Count of records (transactions)
}).reset_index()

df_monthly.rename(columns={'date': 'record_count'}, inplace=True)

print(f"✓ Monthly aggregation complete")
print(f"Original records: {len(df):,}")
print(f"Monthly aggregated records: {len(df_monthly):,}")
print(f"\nSample monthly data:")
df_monthly.head(10)

In [ ]:
# Create district-level monthly aggregation (for analysis)
df_monthly_district = df.groupby(['year_month', 'state', 'district']).agg({
    'demo_age_5_17': 'sum',
    'demo_age_17_': 'sum',
    'date': 'count',
    'pincode': 'nunique'  # Number of unique pincodes
}).reset_index()

df_monthly_district.rename(columns={
    'date': 'record_count',
    'pincode': 'pincode_count'
}, inplace=True)

print(f"District-level monthly records: {len(df_monthly_district):,}")
print(f"\nSample district monthly data:")
df_monthly_district.head(10)

In [ ]:
# Create state-level monthly aggregation
df_monthly_state = df.groupby(['year_month', 'state']).agg({
    'demo_age_5_17': 'sum',
    'demo_age_17_': 'sum',
    'date': 'count',
    'district': 'nunique',
    'pincode': 'nunique'
}).reset_index()

df_monthly_state.rename(columns={
    'date': 'record_count',
    'district': 'district_count',
    'pincode': 'pincode_count'
}, inplace=True)

print(f"State-level monthly records: {len(df_monthly_state):,}")
print(f"\nSample state monthly data:")
df_monthly_state.head(10)

## 8. Create Derived Features

In [ ]:
# Add derived features to monthly datasets
print("Creating derived features...")

# For pincode-level data
df_monthly['total_updates'] = df_monthly['demo_age_5_17'] + df_monthly['demo_age_17_']
df_monthly['child_update_share'] = (df_monthly['demo_age_5_17'] / 
                                     (df_monthly['total_updates'] + 0.0001)) * 100  # Avoid division by zero

# For district-level data
df_monthly_district['total_updates'] = df_monthly_district['demo_age_5_17'] + df_monthly_district['demo_age_17_']
df_monthly_district['child_update_share'] = (df_monthly_district['demo_age_5_17'] / 
                                              (df_monthly_district['total_updates'] + 0.0001)) * 100

# For state-level data
df_monthly_state['total_updates'] = df_monthly_state['demo_age_5_17'] + df_monthly_state['demo_age_17_']
df_monthly_state['child_update_share'] = (df_monthly_state['demo_age_5_17'] / 
                                           (df_monthly_state['total_updates'] + 0.0001)) * 100

print("✓ Derived features created:")
print("  - total_updates")
print("  - child_update_share")

print("\nSample with derived features:")
df_monthly_district[['year_month', 'state', 'district', 'demo_age_5_17', 
                     'demo_age_17_', 'total_updates', 'child_update_share']].head()

## 9. Final Data Summary & Save

In [ ]:
# Final summary statistics
print("=" * 70)
print("FINAL DATA PREPARATION SUMMARY")
print("=" * 70)

print("\n1. RAW DATASET (Daily Records):")
print(f"   Total Records: {len(df):,}")
print(f"   Date Range: {df['date'].min()} to {df['date'].max()}")
print(f"   Unique States: {df['state'].nunique()}")
print(f"   Unique Districts: {df['district'].nunique()}")
print(f"   Unique Pincodes: {df['pincode'].nunique()}")

print("\n2. MONTHLY AGGREGATED DATASETS:")
print(f"   Pincode-level: {len(df_monthly):,} records")
print(f"   District-level: {len(df_monthly_district):,} records")
print(f"   State-level: {len(df_monthly_state):,} records")

print("\n3. DEMOGRAPHIC STATISTICS:")
print(f"   Total child updates (5-17): {df['demo_age_5_17'].sum():,}")
print(f"   Total adult updates (17+): {df['demo_age_17_'].sum():,}")
print(f"   Total updates: {(df['demo_age_5_17'].sum() + df['demo_age_17_'].sum()):,}")

overall_child_share = (df['demo_age_5_17'].sum() / 
                       (df['demo_age_5_17'].sum() + df['demo_age_17_'].sum()) * 100)
print(f"   Overall child update share: {overall_child_share:.2f}%")

print("\n4. DATA QUALITY:")
print(f"   Rows with valid dates: {(~df['date'].isnull()).sum():,}")
print(f"   Rows with valid pincodes: {df['pincode_valid'].sum():,}")
print(f"   Missing values: {df.isnull().sum().sum()}")

In [ ]:
# Save processed datasets
print("\nSaving processed datasets...")

output_dir = '../outputs/'
os.makedirs(output_dir, exist_ok=True)

# Save as CSV
df.to_csv(f'{output_dir}df_clean_daily.csv', index=False)
df_monthly.to_csv(f'{output_dir}df_monthly_pincode.csv', index=False)
df_monthly_district.to_csv(f'{output_dir}df_monthly_district.csv', index=False)
df_monthly_state.to_csv(f'{output_dir}df_monthly_state.csv', index=False)

print("✓ Saved CSV files:")
print(f"  - df_clean_daily.csv ({len(df):,} rows)")
print(f"  - df_monthly_pincode.csv ({len(df_monthly):,} rows)")
print(f"  - df_monthly_district.csv ({len(df_monthly_district):,} rows)")
print(f"  - df_monthly_state.csv ({len(df_monthly_state):,} rows)")

# Also save as pickle for faster loading
df.to_pickle(f'{output_dir}df_clean_daily.pkl')
df_monthly.to_pickle(f'{output_dir}df_monthly_pincode.pkl')
df_monthly_district.to_pickle(f'{output_dir}df_monthly_district.pkl')
df_monthly_state.to_pickle(f'{output_dir}df_monthly_state.pkl')

print("\n✓ Saved pickle files for faster loading")

In [ ]:
# Display sample of final cleaned data
print("\nSample of cleaned daily data:")
display(df[['date', 'year_month', 'state', 'district', 'pincode', 
            'demo_age_5_17', 'demo_age_17_']].head(10))

print("\nSample of monthly district data:")
display(df_monthly_district[['year_month', 'state', 'district', 
                             'demo_age_5_17', 'demo_age_17_', 
                             'total_updates', 'child_update_share']].head(10))

## ✅ Data Preparation Complete!

### Summary of Accomplishments:

1. ✓ **Loaded all CSV files** from DemographicData folder
2. ✓ **Verified data structure** - confirmed all expected columns present
3. ✓ **Date standardization** - converted to datetime format and extracted features
4. ✓ **District-pincode consistency** - standardized text, validated pincodes
5. ✓ **Missing value handling** - filled/removed as appropriate
6. ✓ **Monthly aggregation** - created pincode, district, and state-level datasets
7. ✓ **Derived features** - calculated total_updates and child_update_share

### Next Steps:
- Proceed to univariate analysis
- Begin exploratory visualization
- Calculate baseline statistics